# Metro Project

## Setup

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

from funcs import *
import geopandas as gpd
import pandas as pd
import numpy as np
import contextily as cx
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
import itertools
from libpysal import weights
import networkx as nx
import pickle
from shapely.geometry import LineString, MultiLineString, Point

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Getting county list

In [15]:
fips_path = "data/state_and_county_fips_master.csv"
md_counties = ["Prince George's County", "Montgomery County"]
va_counties = ["Arlington County", "Alexandria city", "Fairfax County", "Fairfax city", "Falls Church city", "Loudoun County"]
states = ["MD", "DC", "VA"]

md_codes = get_county_codes(fips_path, ["MD"], md_counties)
va_codes = get_county_codes(fips_path, ["VA"], va_counties)
county_shapes_df = load_shapefile("data/counties/c_18mr25.shp")
md_shapes_df = county_shapes_df[county_shapes_df['STATE'].isin(['MD'])]
md_shapes_df = md_shapes_df[md_shapes_df['FIPS'].apply(lambda a: a[2:]).isin(md_codes)]
va_shapes_df = county_shapes_df[county_shapes_df['STATE'].isin(['VA'])]
va_shapes_df = va_shapes_df[va_shapes_df['FIPS'].apply(lambda a: a[2:]).isin(va_codes)]
dc_shape_df = county_shapes_df[county_shapes_df['STATE'].isin(['DC'])]
county_shapes_df = pd.concat([md_shapes_df,va_shapes_df,dc_shape_df])

### Getting Neighborhood Centroids

In [23]:
arl_neighborhoods = load_geojson("data/neighborhoods/Arlington_Neighborhoods_Program_Areas.geojson")[["NEIGHBORHOOD","geometry"]]
arl_neighborhoods["NAME"] = arl_neighborhoods["NEIGHBORHOOD"]
arl_neighborhoods.drop("NEIGHBORHOOD", axis=1, inplace=True)
arl_neighborhoods.geometry = arl_neighborhoods.centroid
md_neighborhoods = load_geojson("data/neighborhoods/Maryland_Census_Designated_Areas_-_Census_Designated_Places_2020.geojson")[["NAME","geometry"]]
md_neighborhoods.geometry = md_neighborhoods.centroid
dc_neighborhoods = load_geojson("data/neighborhoods/neighborhood-names-centroid.geojson")[["NAME","geometry"]]
neighborhoods_df = pd.concat([arl_neighborhoods,md_neighborhoods,dc_neighborhoods]).to_crs(epsg=3857)

C:\Users\srjnk\AppData\Local\Temp\ipykernel_14420\4024320617.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  arl_neighborhoods.geometry = arl_neighborhoods.centroid
C:\Users\srjnk\AppData\Local\Temp\ipykernel_14420\4024320617.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  md_neighborhoods.geometry = md_neighborhoods.centroid


### Getting existing network

In [4]:
'''dcs_df = load_shapefile("data/real_transit/dcs/DC_Streetcar_Routes.shp").to_crs(epsg=3857)
dcs_df["color"] = 'brown'
marc_df = load_shapefile("data/real_transit/marc/Maryland_Transit_-_MARC_Train_Lines.shp").to_crs(epsg=3857)
marc_df["color"] = marc_df["Rail_Name"].apply(lambda a: '#EFAD1D' if 'Brunswick' in a else '#F15828' if 'Camden' in a else '#C71F3E')
wmata_df = load_shapefile("data/real_transit/wmata/Metro_Lines_Regional.shp").to_crs(epsg=3857)
wmata_df["color"] = wmata_df["NAME"].apply(lambda a: 
    '#F9921D' if 'orange' in a else 
    '#A1A3A1' if 'silver' in a else 
    '#E41838' if 'red' in a else
    '#FED201' if 'yellow' in a else
    '#01A850' if 'green' in a else
    '#0077C1')
vre_df = load_shapefile("data/real_transit/vre/Virginia_Railway_Express_Routes.shp").to_crs(epsg=3857)
vre_df["color"] = vre_df["RAILWAY_NM"].apply(lambda a: '#156DB4' if 'Manassas' in a else '#DD3534')
pl_df = load_shapefile("data/real_transit/pl/PurpleLineAlignment.shp").to_crs(epsg=3857)
pl_df["color"] = "#793390"'''

'dcs_df = load_shapefile("data/real_transit/dcs/DC_Streetcar_Routes.shp").to_crs(epsg=3857)\ndcs_df["color"] = \'brown\'\nmarc_df = load_shapefile("data/real_transit/marc/Maryland_Transit_-_MARC_Train_Lines.shp").to_crs(epsg=3857)\nmarc_df["color"] = marc_df["Rail_Name"].apply(lambda a: \'#EFAD1D\' if \'Brunswick\' in a else \'#F15828\' if \'Camden\' in a else \'#C71F3E\')\nwmata_df = load_shapefile("data/real_transit/wmata/Metro_Lines_Regional.shp").to_crs(epsg=3857)\nwmata_df["color"] = wmata_df["NAME"].apply(lambda a: \n    \'#F9921D\' if \'orange\' in a else \n    \'#A1A3A1\' if \'silver\' in a else \n    \'#E41838\' if \'red\' in a else\n    \'#FED201\' if \'yellow\' in a else\n    \'#01A850\' if \'green\' in a else\n    \'#0077C1\')\nvre_df = load_shapefile("data/real_transit/vre/Virginia_Railway_Express_Routes.shp").to_crs(epsg=3857)\nvre_df["color"] = vre_df["RAILWAY_NM"].apply(lambda a: \'#156DB4\' if \'Manassas\' in a else \'#DD3534\')\npl_df = load_shapefile("data/real_trans

In [5]:
'''md_df = load_shapefile("data/md/tl_2023_24_tabblock20.shp")
md_df = md_df[md_df["COUNTYFP20"].isin(md_codes.tolist())].copy()
va_df = load_shapefile("data/va/tl_2023_51_tabblock20.shp")
va_df = va_df[va_df["COUNTYFP20"].isin(va_codes.tolist())].copy()
dc_df = load_shapefile("data/dc/tl_2023_11_tabblock20.shp")
df = pd.concat([md_df, va_df, dc_df], ignore_index=True)
df = gpd.GeoDataFrame(df)
df.to_crs("EPSG:4326", inplace=True)
df["SID"] = df.index
df["INTPTLON20"] = df["INTPTLON20"].astype(float)
df["INTPTLAT20"] = df["INTPTLAT20"].astype(float)
df["NEIGHBORS"] = None
df = compute_transit_potential(df)
save_geojson(df, "data/complete_region_df.geojson")
df = load_geojson("data/complete_region_df.geojson")
dc_df = load_shapefile("data/dc/tl_2023_11_tabblock20.shp")
extremities = [df["INTPTLON20"].min(), df["INTPTLAT20"].min(), df["INTPTLON20"].max(), df["INTPTLAT20"].max()]
extremities_dc = [dc_df["INTPTLON20"].min(), dc_df["INTPTLAT20"].min(), dc_df["INTPTLON20"].max(), dc_df["INTPTLAT20"].max()]
df_map = df.to_crs(epsg=3857)
ex_map = np.array([df_map.centroid.x.min(),df_map.centroid.y.min(),df_map.centroid.x.max(),df_map.centroid.y.max()])
np.save("data/ex_map.npy", ex_map)
df_map_dc = dc_df.to_crs(epsg=3857)
ex_map_dc = np.array([df_map_dc.centroid.x.min(),df_map_dc.centroid.y.min(),df_map_dc.centroid.x.max(),df_map_dc.centroid.y.max()])
np.save("data/ex_map_dc.npy", ex_map_dc)

ex_map = np.load("data/ex_map.npy")
ex_map_dc = np.load("data/ex_map_dc.npy")
dcs_stations_df = load_geojson("data/real_transit/dcs/dc-streetcar-stops.geojson")
marc_stations_df = load_geojson("data/real_transit/marc/Maryland_Transit_-_MARC_Train_Stations.geojson")
pl_stations_df = load_geojson("data/real_transit/pl/Purple_Line_Stations.geojson")
vre_stations_df = load_geojson("data/real_transit/vre/vre-stations.geojson")
wmata_stations_df = load_geojson("data/real_transit/wmata/Metro_Stations_Regional.geojson")
mc_stations_df = load_geojson("data/real_transit/mc/Maryland_Local_Transit_-_Montgomery_County_Ride_On_Stops.geojson")
mta_stations_df = load_geojson("data/real_transit/mta/Maryland_Transit_-_MTA_Bus_Stops.geojson")
pgc_stations_df = load_geojson("data/real_transit/pgc/Maryland_Local_Transit_-_Prince_Georges_County_Transit_Stops.geojson")
wmatabus_stations_df = load_geojson("data/real_transit/wmatabus/Metro_Bus_Stops.geojson")
vbus_stations_df = filter_points_in_polygons(load_geojson("data/real_transit/vbus/virginia_bus_stops.geojson"), county_shapes_df.geometry)
points_gdf = pd.concat([dcs_stations_df.geometry, 
                        marc_stations_df.geometry,
                        pl_stations_df.geometry, 
                        vre_stations_df.geometry, 
                        wmata_stations_df.geometry,
                        mc_stations_df.geometry,
                        mta_stations_df.geometry,
                        pgc_stations_df.geometry,
                        wmatabus_stations_df.geometry
])
points_gdf = filter_points_in_polygons(points_gdf, county_shapes_df.geometry).to_crs(epsg=3857)

# Drop duplicates based on x and y
points_gdf = points_gdf.drop_duplicates().reset_index(drop=True)

save_geojson(gpd.GeoDataFrame(points_gdf),"data/transit.geojson")'''

'md_df = load_shapefile("data/md/tl_2023_24_tabblock20.shp")\nmd_df = md_df[md_df["COUNTYFP20"].isin(md_codes.tolist())].copy()\nva_df = load_shapefile("data/va/tl_2023_51_tabblock20.shp")\nva_df = va_df[va_df["COUNTYFP20"].isin(va_codes.tolist())].copy()\ndc_df = load_shapefile("data/dc/tl_2023_11_tabblock20.shp")\ndf = pd.concat([md_df, va_df, dc_df], ignore_index=True)\ndf = gpd.GeoDataFrame(df)\ndf.to_crs("EPSG:4326", inplace=True)\ndf["SID"] = df.index\ndf["INTPTLON20"] = df["INTPTLON20"].astype(float)\ndf["INTPTLAT20"] = df["INTPTLAT20"].astype(float)\ndf["NEIGHBORS"] = None\ndf = compute_transit_potential(df)\nsave_geojson(df, "data/complete_region_df.geojson")\ndf = load_geojson("data/complete_region_df.geojson")\ndc_df = load_shapefile("data/dc/tl_2023_11_tabblock20.shp")\nextremities = [df["INTPTLON20"].min(), df["INTPTLAT20"].min(), df["INTPTLON20"].max(), df["INTPTLAT20"].max()]\nextremities_dc = [dc_df["INTPTLON20"].min(), dc_df["INTPTLAT20"].min(), dc_df["INTPTLON20"].max

In [6]:
'''df["point_likelihood"] = df["transit_potential"]
l = list(set(get_points(df, extremities)))
points = gpd.GeoDataFrame(df[df['SID'].isin(l)])
save_geojson(points, "data/graph_points.geojson")
points = load_geojson("data/graph_points.geojson")
combined_df_dc = load_geojson("data/dc/non-population-points/combined_df.geojson")
combined_df_md = load_geojson("data/md/non-population-points/combined_df.geojson")
combined_df_va = load_geojson("data/va/non-population-points/combined_df.geojson")
points = reset_and_concat(points, combined_df_dc, combined_df_md, combined_df_va)
points = points["geometry"]
df_points = gpd.GeoDataFrame(geometry=points.centroid).drop_duplicates().reset_index(drop=True)
df_points = pd.concat([df_points.to_crs(3857), load_geojson("data/transit.geojson")]).reset_index(drop=True)
save_geojson(df_points, "data/complete_points.geojson")'''
df_points = load_geojson("data/complete_points.geojson")

## Create Network

### KDE Heatmap

In [7]:
'''kde = plot_kde_heatmap(df_points)'''

'kde = plot_kde_heatmap(df_points)'

### Gabriel Network

In [8]:
'''pts_array = np.array(list(zip(df_points.geometry.x, df_points.geometry.y)))
gabriel = weights.Gabriel.from_dataframe(df_points, use_index=True, silence_warnings=True)
network = gabriel.to_networkx()

gabriel_contracted, new_positions = contract_louvain_communities_with_positions(
    network, {n: pts_array[n] for n in network.nodes()}, 0.07
)
gabriel_contracted = remove_isolated_nodes(gabriel_contracted)'''

'pts_array = np.array(list(zip(df_points.geometry.x, df_points.geometry.y)))\ngabriel = weights.Gabriel.from_dataframe(df_points, use_index=True, silence_warnings=True)\nnetwork = gabriel.to_networkx()\n\ngabriel_contracted, new_positions = contract_louvain_communities_with_positions(\n    network, {n: pts_array[n] for n in network.nodes()}, 0.07\n)\ngabriel_contracted = remove_isolated_nodes(gabriel_contracted)'

In [9]:
'''assign_edge_weights(gabriel_contracted, new_positions)
radius=1000
assign_node_scores(gabriel_contracted, new_positions, kde, radius)
save_graph_to_geojson(gabriel_contracted, new_positions, "data/output/network.geojson")
with open("pickle/kde.pkl", "wb") as f:   
    pickle.dump(kde, f)
with open("pickle/graph.pkl", "wb") as f:   
    pickle.dump(gabriel_contracted, f)
with open("pickle/positions.pkl", "wb") as f:   
    pickle.dump(new_positions, f)
%sx scp pickle/*.pkl spencerrjenkins@ssh.ocf.berkeley.edu:~/cmsc725_wmata_map/pickle'''

'assign_edge_weights(gabriel_contracted, new_positions)\nradius=1000\nassign_node_scores(gabriel_contracted, new_positions, kde, radius)\nsave_graph_to_geojson(gabriel_contracted, new_positions, "data/output/network.geojson")\nwith open("pickle/kde.pkl", "wb") as f:   \n    pickle.dump(kde, f)\nwith open("pickle/graph.pkl", "wb") as f:   \n    pickle.dump(gabriel_contracted, f)\nwith open("pickle/positions.pkl", "wb") as f:   \n    pickle.dump(new_positions, f)\n%sx scp pickle/*.pkl spencerrjenkins@ssh.ocf.berkeley.edu:~/cmsc725_wmata_map/pickle'

## Naive Network

In [10]:
'''lines, _,_ = perform_walks(
    gabriel_contracted, new_positions, num_walks=20, 
    min_distance=45000, max_distance=100000, traversed_edges=set(), complete_traversed_edges=[])
#ax=plot_network(gabriel_contracted, new_positions, ex_map, width=0.5, alpha=0.5)
#ax=plot_walks(gabriel_contracted, new_positions, lines, ax, kde, ex_map)
groups = group_assigner(lines,gabriel_contracted, new_positions, threshold=0.5)
status = mark_station_nodes(lines,gabriel_contracted,new_positions,min_station_dist=1000, groups=groups)
names = assign_station_neighborhoods(new_positions, status, neighborhoods_df)
save_lines_to_geojson(lines, gabriel_contracted, new_positions, kde, "data/output/lines_naive.geojson", status, groups, names)'''

'lines, _,_ = perform_walks(\n    gabriel_contracted, new_positions, num_walks=20, \n    min_distance=45000, max_distance=100000, traversed_edges=set(), complete_traversed_edges=[])\n#ax=plot_network(gabriel_contracted, new_positions, ex_map, width=0.5, alpha=0.5)\n#ax=plot_walks(gabriel_contracted, new_positions, lines, ax, kde, ex_map)\ngroups = group_assigner(lines,gabriel_contracted, new_positions, threshold=0.5)\nstatus = mark_station_nodes(lines,gabriel_contracted,new_positions,min_station_dist=1000, groups=groups)\nnames = assign_station_neighborhoods(new_positions, status, neighborhoods_df)\nsave_lines_to_geojson(lines, gabriel_contracted, new_positions, kde, "data/output/lines_naive.geojson", status, groups, names)'

In [11]:
'''lines, traversed_edges, complete_traversed_edges = perform_walks(
    gabriel_contracted, new_positions, num_walks=20, 
    min_distance=45000, max_distance=100000, traversed_edges=set(), complete_traversed_edges=[])
for i in range(100):
    lines, traversed_edges, complete_traversed_edges = replace_lowest_scoring_walk(
        lines, new_positions, kde, gabriel_contracted, traversed_edges, complete_traversed_edges, 
        min_distance=45000, max_distance=100000, radius=radius)
#ax=plot_network(gabriel_contracted, new_positions, ex_map, width=0.5, alpha=0.5)
#ax=plot_walks(gabriel_contracted, new_positions, lines, ax, kde, ex_map)
groups = group_assigner(lines,gabriel_contracted, new_positions, threshold=0.5)
status = mark_station_nodes(lines,gabriel_contracted,new_positions,min_station_dist=1000, groups=groups)
names = assign_station_neighborhoods(new_positions, status, neighborhoods_df)
save_lines_to_geojson(lines, gabriel_contracted, new_positions, kde, "data/output/lines_iterative.geojson", status, groups, names)'''

'lines, traversed_edges, complete_traversed_edges = perform_walks(\n    gabriel_contracted, new_positions, num_walks=20, \n    min_distance=45000, max_distance=100000, traversed_edges=set(), complete_traversed_edges=[])\nfor i in range(100):\n    lines, traversed_edges, complete_traversed_edges = replace_lowest_scoring_walk(\n        lines, new_positions, kde, gabriel_contracted, traversed_edges, complete_traversed_edges, \n        min_distance=45000, max_distance=100000, radius=radius)\n#ax=plot_network(gabriel_contracted, new_positions, ex_map, width=0.5, alpha=0.5)\n#ax=plot_walks(gabriel_contracted, new_positions, lines, ax, kde, ex_map)\ngroups = group_assigner(lines,gabriel_contracted, new_positions, threshold=0.5)\nstatus = mark_station_nodes(lines,gabriel_contracted,new_positions,min_station_dist=1000, groups=groups)\nnames = assign_station_neighborhoods(new_positions, status, neighborhoods_df)\nsave_lines_to_geojson(lines, gabriel_contracted, new_positions, kde, "data/output/l

## Genetic

In [12]:
'''%sx scp spencerrjenkins@ssh.ocf.berkeley.edu:~/cmsc725_wmata_map/pickle/*.pkl ./pickle/'''
'''with open("pickle/graph.pkl", "rb") as f:   
    gabriel_contracted = pickle.load(f)
with open("pickle/positions.pkl", "rb") as f:   
    new_positions = pickle.load(f)
with open("pickle/kde.pkl", "rb") as f:   
    kde = pickle.load(f)
with open("pickle/ex_map_dc.pkl", "rb") as f:   
    ex_map_dc = pickle.load(f)'''
'''with open("pickle/best_routes.pkl", "rb") as f:   
    best_routes = pickle.load(f)
with open("pickle/best_score.pkl", "rb") as f:   
    best_score = pickle.load(f)
with open("pickle/log.pkl", "rb") as f:   
    log = pickle.load(f)
groups = group_assigner(best_routes,gabriel_contracted, new_positions, threshold=0.3)
status = mark_station_nodes(best_routes,gabriel_contracted,new_positions,min_station_dist=1000, groups=groups)
names = assign_station_neighborhoods(new_positions, status, neighborhoods_df)
save_lines_to_geojson(best_routes, gabriel_contracted, new_positions, kde, "data/output/lines_genetic.geojson", status, groups, names)'''

'with open("pickle/best_routes.pkl", "rb") as f:   \n    best_routes = pickle.load(f)\nwith open("pickle/best_score.pkl", "rb") as f:   \n    best_score = pickle.load(f)\nwith open("pickle/log.pkl", "rb") as f:   \n    log = pickle.load(f)\ngroups = group_assigner(best_routes,gabriel_contracted, new_positions, threshold=0.3)\nstatus = mark_station_nodes(best_routes,gabriel_contracted,new_positions,min_station_dist=1000, groups=groups)\nnames = assign_station_neighborhoods(new_positions, status, neighborhoods_df)\nsave_lines_to_geojson(best_routes, gabriel_contracted, new_positions, kde, "data/output/lines_genetic.geojson", status, groups, names)'

In [49]:
with open("pickle/graph.pkl", "rb") as f:
    gabriel_contracted = pickle.load(f)
with open("pickle/positions.pkl", "rb") as f:
    new_positions = pickle.load(f)
with open("pickle/kde.pkl", "rb") as f:
    kde = pickle.load(f)
with open("pickle/ex_map_dc.pkl", "rb") as f:
    ex_map_dc = pickle.load(f)
points = load_geojson("data/graph_points.geojson")
combined_df_dc = load_geojson("data/dc/non-population-points/combined_df.geojson")
combined_df_md = load_geojson("data/md/non-population-points/combined_df.geojson")
combined_df_va = load_geojson("data/va/non-population-points/combined_df.geojson")
points = reset_and_concat(points, combined_df_dc, combined_df_md, combined_df_va)
points = points["geometry"]
points = points.to_crs(epsg=3857)
df_points = gpd.GeoDataFrame(geometry=points.centroid).drop_duplicates().reset_index(drop=True)
lines_naive, status_naive, groups_naive, names_naive = load_lines_from_geojson("./data/output/lines_naive.geojson")
lines_iter, status_iter, groups_iter, names_iter = load_lines_from_geojson("./data/output/lines_iterative.geojson")
lines_genetic, status_genetic, groups_genetic, names_genetic = load_lines_from_geojson("./data/output/lines_genetic.geojson")
wmata_stations_df = load_geojson("data/real_transit/wmata/Metro_Stations_Regional.geojson")
blocks = load_geojson("data/complete_region_df.geojson").to_crs(epsg=3857)
popkde = population_density_kde(blocks)

In [62]:
from funcs import *
a=gpd.GeoDataFrame(geometry=list(Point(i) if status_genetic[i] else Point([1,1]) for i in status_genetic))
a.crs='4326'
a = a.to_crs(epsg=3857)
station_gdf_catchment_coverage(
    a,
    df_points), station_gdf_catchment_coverage(
    a, neighborhoods_df),average_distance_to_points_within_polygon(
    a,combine_polygons_to_single(county_shapes_df.to_crs(epsg=3857))),average_distance_to_points_within_polygon(
    a,dc_shape_df.to_crs(epsg=3857).iloc[0].geometry)

(np.float64(17.471108089734873),
 np.float64(6.6115702479338845),
 11028.791073538747,
 916.2676772935321)

In [ ]:
a=gpd.GeoDataFrame(geometry=list(Point(i) if status_iter[i] else Point([1,1]) for i in status_iter))
a.crs='4326'
a = a.to_crs(epsg=3857)
station_gdf_catchment_coverage(
    a,
    df_points), station_gdf_catchment_coverage(
    a, neighborhoods_df),average_distance_to_points_within_polygon(
    a,combine_polygons_to_single(county_shapes_df.to_crs(epsg=3857))),average_distance_to_points_within_polygon(
    a,dc_shape_df.to_crs(epsg=3857).iloc[0].geometry), estimate_population_in_catchments(
    popkde, a, catchment_radius=500, grid_resolution=100)

(np.float64(24.1162474507138),
 np.float64(4.958677685950414),
 11193.393076589782,
 1667.2053960291078)

In [ ]:
a=gpd.GeoDataFrame(geometry=list(Point(i) if status_naive[i] else Point([1,1]) for i in status_naive))
a.crs='4326'
a = a.to_crs(epsg=3857)
station_gdf_catchment_coverage(
    a,
    df_points), station_gdf_catchment_coverage(
    a, neighborhoods_df),average_distance_to_points_within_polygon(
    a,combine_polygons_to_single(county_shapes_df.to_crs(epsg=3857))),average_distance_to_points_within_polygon(
    a,dc_shape_df.to_crs(epsg=3857).iloc[0].geometry), estimate_population_in_catchments(
    popkde, a, catchment_radius=500, grid_resolution=100)

(np.float64(18.949694085656017),
 np.float64(3.994490358126722),
 10856.134057332005,
 1586.2258374218466)

In [ ]:
a=wmata_stations_df.to_crs(epsg=3857)
a.crs='4326'
a = a.to_crs(epsg=3857)
station_gdf_catchment_coverage(
    a,
    df_points), station_gdf_catchment_coverage(
    a, neighborhoods_df),average_distance_to_points_within_polygon(
    a,combine_polygons_to_single(county_shapes_df.to_crs(epsg=3857))),average_distance_to_points_within_polygon(
    a,dc_shape_df.to_crs(epsg=3857).iloc[0].geometry), estimate_population_in_catchments(
    popkde, a, catchment_radius=500, grid_resolution=100)

(np.float64(13.392250169952414),
 np.float64(3.0303030303030303),
 14940.644861813917,
 1592.1486479335385)

In [54]:
estimate_population_in_catchments(
    popkde, a, catchment_radius=500, grid_resolution=10)

np.float64(0.06800970470971124)

In [60]:
popkde.score_samples([[a.iloc[0].geometry.x, a.iloc[0].geometry.y]])

array([-21.97564391])